<a href="https://colab.research.google.com/github/DongDong-Zoez/pytorchAI/blob/main/Transformer/ViT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [ViT](https://arxiv.org/pdf/2010.11929.pdf)

Transformer 是 NLP 領域中的一個里程碑，我們開始關心能不能把在 NLP 表現最好的 Transformer 模型應用到電腦視覺領域。ViT 是一個把 Transformer 引入 CV 領域的架構，我們就來看看 ViT 到底做了什麼能用一個架構做兩個領域的事情吧

## Embedded Patches

![](https://miro.medium.com/max/1400/1*l37va2Mu8Snx6LLb13430A.png)

首先我們知道序列是一串向量，向量是 1D 的，但是影像是 2D 的，我們需要找一個方法把影響從 2D 映射到 1D，一個很直覺的想法是把影像拉成向量後放入 Transformer 訓練，但是這樣計算量太大了，試想一個 224$\times$224 的影像，拉成向量就是 50176 這麼高維度，Transformer 雖然理論上可以負擔任意長度的向量，但是這也導致計算困難。

ViT 的作者提出了另一個想法，把影像分割成一個個小塊 (patch)，每個小塊就可以看成是 NLP 序列中的 token，因為被分割成小塊，所以計算量也相對減少。舉例來說，一張 224$\times$224 的影像，我們每有一個 16$\times$16 區域，就分成一個 patch，我們的影像就總共被分成 $\frac{224\times 224}{16\times 16}=196$ 個 patch，每個 patch 大小為 16$\times$16，這就等於 NLP 中有 196 個 token，每個 token 都 embedding 到 256 維度，這個過程就稱為 Embedded Patches



## Position Embedding

在 NLP 任務中，我們知道一個句子中的單詞順序調換會影響閱讀，因此我們會對其加入 Position Embedding，同樣的，如果把一張影像分成若干小塊，重新排列組合後的影像也會是一張新的影像，所以我們也會加入 Position Embedding

作者試了不同的 Position Embedding 方法，有 1D、2D 絕對位置編碼，也有利用相對位置編碼的，結果其實都差不多，但是比起不加入 Position Embedding 結果好上不少

為什麼不同的編碼對結果沒有什麼作用呢? 作者給出的解釋是，直接在 patch 上做排列組合要比在原解析度上做排列組合要容易的多，所以也不需要太多的位置資訊

## cls token

ViT 借鑑了 BERT 的想法，在 196 個 patch 上加上了一個 cls token，這個 token 用來濃縮其他 patch 的信息，進而做出模型預測

cls token 一開始被模型初始化為 0，之後會經過模型訓練，最後用來預測

作者研究發現如果訓練個夠多跟參數調整上可以不用加上 cls token，只用 GAP 也能達到差不多的效果

## Forward

我們整合一下上述的概念，模型整體的算法如下

1. 將 224$\times$224$\times$3 影像分割成 196 個 16$\times$16$\times$3 patch
2. 將 196 個 patch 乘上一個線性變換矩陣，即 $[x_p^1\mbox{E},x_p^2\mbox{E},\cdots,x_p^N\mbox{E}]$
3. 加上 cls token 和 Position Embedding，即 $\mbox{z}_0=[x_{\text{class}},x_p^1\mbox{E},x_p^2E,\cdots,x_p^N\mbox{E}]+\mbox{E}_{pos}$ (大小為 197$\times$768)
4. 通過 Encoder L 次，其中會執行以下內容
  - 通過 LN 和 MSA，即 $\mbox{z}_l'=\mbox{MSA}(\mbox{LA}(\mbox{z}_{l-1}))+\mbox{z}_{l-1}$
  - 通過 LN 和 MLP，即 $\mbox{z}_l=\mbox{MLP}(\mbox{LA}(\mbox{z}'_{l-1}))+\mbox{z}'_{l-1}$
5. 最後捨棄所有 patch 輸出，只用 cls token 通過 MLP 後做 softmax 預測

以下是一個簡易版本的 MLP pytorch 代碼釋例

In [1]:
import torch.nn as nn

class MLP(nn.Module):

    def __init__(self, embed_size=768, expansion_dim=4):
        super().__init__()

        self.linear = nn.Sequential(
            nn.Linear(embed_size, embed_size*expansion_dim),
            nn.GELU(inplace=True),
            nn.Linear(embed_size*expansion_dim, embed_size)
        )

    def forward(self, x):

        x = self.linear(x)

        return x

## HybridViT

混合 ViT 是一種把 CNN 和 ViT 結合的方法，我們在前文說過 ViT 把ㄌ224$\times$224$\times$3 的影像分割成 196 個長度為 16$\times$16$\times$3 的 patch，每個 patch 的大小是 14$\times$14

HybridViT 用了另一種方法得到大小為 16$\times$16$\times$3 的 patch，我們用一個 CNN 網路提取特徵圖，接上一個 kernel size 和 padding 皆為 14 的 2D-Convolution，其輸出通道數為 patch embedding size，在把特徵圖拉成 vector，最後的輸出大小為 [bs, number of patch, patch embedding size]

patch embedding size:
  - tiny: 192
  - small: 384
  - base: 768
  - large: 1024

In [8]:
# The code copy from https://github.com/rwightman/pytorch-image-models/blob/cd34913278f8511ba53492ed186b4e08f890add6/timm/models/vision_transformer_hybrid.py
def to_2tuple(x):
    return (x, x)

class HybridEmbed(nn.Module):
    """ CNN Feature Map Embedding
    Extract feature map from CNN, flatten, project to embedding dim.
    """
    def __init__(self, backbone, img_size=224, patch_size=1, feature_size=None, in_chans=3, embed_dim=768):
        super().__init__()
        assert isinstance(backbone, nn.Module)
        img_size = to_2tuple(img_size)
        patch_size = to_2tuple(patch_size)
        self.img_size = img_size
        self.patch_size = patch_size
        self.backbone = backbone
        if feature_size is None:
            with torch.no_grad():
                # NOTE Most reliable way of determining output dims is to run forward pass
                training = backbone.training
                if training:
                    backbone.eval()
                o = self.backbone(torch.zeros(1, in_chans, img_size[0], img_size[1]))
                if isinstance(o, (list, tuple)):
                    o = o[-1]  # last feature if backbone outputs list/tuple of features
                feature_size = o.shape[-2:]
                feature_dim = o.shape[1]
                backbone.train(training)
        else:
            feature_size = to_2tuple(feature_size)
            if hasattr(self.backbone, 'feature_info'):
                feature_dim = self.backbone.feature_info.channels()[-1]
            else:
                feature_dim = self.backbone.num_features
        assert feature_size[0] % patch_size[0] == 0 and feature_size[1] % patch_size[1] == 0
        self.grid_size = (feature_size[0] // patch_size[0], feature_size[1] // patch_size[1])
        self.num_patches = self.grid_size[0] * self.grid_size[1]
        self.proj = nn.Conv2d(feature_dim, embed_dim, kernel_size=patch_size, stride=patch_size)

    def forward(self, x):
        x = self.backbone(x)
        print(x.shape)
        if isinstance(x, (list, tuple)):
            x = x[-1]  # last feature if backbone outputs list/tuple of features
        x = self.proj(x).flatten(2).transpose(1, 2)
        return x

In [10]:
import torchvision
import torch
m = torchvision.models.vgg16().features
h = HybridEmbed(m)
x = torch.randn((1,3,224,224))
h(x).shape

torch.Size([1, 512, 7, 7])


torch.Size([1, 49, 768])

## Input size

如果你要使用 ViT 模型，盡量把影像大小插值為 ViT 預訓練大小，因為如果影像解析度變大，你維持相同的 patch size，這樣 patch 數量會增多，位置編碼也不再有意義

In [13]:
#!pip install timm
import timm
m = timm.models.vit_tiny_r_s16_p8_224(pretrained=True)
m(x).shape

torch.Size([1, 1000])

## Disscussion

ViT 這篇論文的方法只應用於影像分類任務上，但也確實做到了 state-of-the-art。作者在實驗過程中發現，在少量的數據訓練中，ResNet 的表現要好於 ViT，這是因為 Convolution 的運算中含有 inductive biases

inductive biases 意思為模型運算的先驗資訊，例如在 Convolution 運算中就有兩個先驗資訊

1. locality
2. translation equivariance

第一個表示空間越鄰近的物體通常特徵也越相似，所以 Convolution 只 focus 在同一個感受野內的物件，而 Attention 是全局考慮的

第二個表示轉換的不變性，捲機運算與平移運算可以相互交換不影響結果，你可以想像一張影像的物體就算移到影像的其他位置，我們用 Convolution 偵測出來的特徵依然會一樣

這兩個 inductive biases 就是 Convolution 的強力假設，有了這些強力假設，相對於 ViT，ResNet 就可以使用相對少的影像做訓練，這也是為什麼 Transformer 模型可以做大很大的網路結構又不過擬合，或者用很多影像訓練還會提升表現，因為模型沒有什麼先驗的假測，所以模型學習的很慢

